In [35]:
import pandas as pd

# Read CSV files and set DataFrame names accordingly
sf_bonus = pd.read_csv(r"D:\STRATASCRATCH 1\City of San Francisco\sf_bonus.csv")
sf_crime_incidents_2014_01 = pd.read_csv(r"D:\STRATASCRATCH 1\City of San Francisco\sf_crime_incidents_2014_01.csv")
sf_employee = pd.read_csv(r"D:\STRATASCRATCH 1\City of San Francisco\sf_employee.csv")
sf_public_salaries = pd.read_csv(r"D:\STRATASCRATCH 1\City of San Francisco\sf_public_salaries.csv")
sf_restaurant_health_violations = pd.read_csv(r"D:\STRATASCRATCH 1\City of San Francisco\sf_restaurant_health_violations.csv")
library_usage = pd.read_csv(r"D:\STRATASCRATCH 1\City of San Francisco\library_usage.csv")

### 1 -Make a pivot table to find the highest payment in each year for each employee.
Find payment details for 2011, 2012, 2013, and 2014.
Output payment details along with the corresponding employee name.
Order records by the employee name in ascending order

In [3]:
sf_public_salaries.columns

Index(['id', 'employeename', 'jobtitle', 'basepay', 'overtimepay', 'otherpay',
       'benefits', 'totalpay', 'totalpaybenefits', 'year', 'notes', 'agency',
       'status'],
      dtype='object')

In [4]:
# Create a pivot table for highest payment per employee by year
pivot_table = sf_public_salaries.pivot_table(
    index='employeename', 
    columns='year', 
    values='totalpay', 
    aggfunc='max'
)

# Filter for the years 2011 to 2014 and reset the index
result = pivot_table.loc[:, [2011, 2012, 2013, 2014]].reset_index()

# Sort by employee name
result = result.sort_values(by='employeename')

# Output the result
result.head()


year,employeename,2011,2012,2013,2014
0,ANNA BROWN,238551.88,NaN,NaN,NaN
1,Aaron Schmidt,NaN,91903.92,NaN,NaN
2,Adina M Diamond,NaN,NaN,NaN,58496.19
3,Alan K Tolbert,NaN,NaN,NaN,2070.29
4,Alexander M Lamond,NaN,NaN,NaN,106319.44


### 2 -

In [6]:
sf_crime_incidents_2014_01.columns

Index(['incidnt_num', 'category', 'descript', 'day_of_week', 'date', 'time',
       'pd_district', 'resolution', 'address', 'lon', 'lat', 'location', 'id'],
      dtype='object')

### 2 -Find the number of crime occurrences for each day of the week.
Output the day alongside the corresponding crime count.

In [7]:
# Count crime occurrences for each day of the week
crime_counts = sf_crime_incidents_2014_01['day_of_week'].value_counts().reset_index()
crime_counts.columns = ['day_of_week', 'crime_count']

# Sort by day_of_week
crime_counts = crime_counts.sort_values(by='day_of_week')

# Output the result
crime_counts.head()


,day_of_week,crime_count
5,Friday,12
3,Monday,14
1,Saturday,18
6,Sunday,9
2,Thursday,16


### 3 -Find the number of words in each business name. Avoid counting special symbols as words (e.g. &). Output the business name and its count of words.

In [9]:
sf_restaurant_health_violations.columns

Index(['business_id', 'business_name', 'business_address', 'business_city',
       'business_state', 'business_postal_code', 'business_latitude',
       'business_longitude', 'business_location', 'business_phone_number',
       'inspection_id', 'inspection_date', 'inspection_score',
       'inspection_type', 'violation_id', 'violation_description',
       'risk_category'],
      dtype='object')

In [10]:
import pandas as pd

# Assuming sf_restaurant_health_violations is already loaded as a DataFrame
# Count words in business names, ignoring special symbols
sf_restaurant_health_violations['word_count'] = sf_restaurant_health_violations['business_name'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.split().str.len()

# Select business name and word count
result = sf_restaurant_health_violations[['business_name', 'word_count']]

# Output the result DataFrame
result.head()


,business_name,word_count
0,John Chin Elementary School,4
1,Sutter Pub and Restaurant,4
2,SRI THAI CUISINE,3
3,Washington Bakery & Restaurant,3
4,Brothers Restaurant,2


### 4 -Find the number of inspections that resulted in each risk category per each inspection type.
Consider the records with no risk category value belongs to a separate category.
Output the result along with the corresponding inspection type and the corresponding total number of inspections per that type. The output should be pivoted, meaning that each risk category + total number should be a separate column.
Order the result based on the number of inspections per inspection type in descending order.

In [11]:
import pandas as pd

# Group by inspection type and risk category, counting inspections
inspections_count = sf_restaurant_health_violations.groupby(['inspection_type', 'risk_category']).size().reset_index(name='count')

# Replace NaN in risk_category with a separate category
inspections_count['risk_category'] = inspections_count['risk_category'].fillna('No Risk Category')

# Pivot the table
pivot_result = inspections_count.pivot_table(index='inspection_type', columns='risk_category', values='count', fill_value=0)

# Add total inspections column
pivot_result['total_inspections'] = pivot_result.sum(axis=1)

# Sort by total inspections in descending order
pivot_result = pivot_result.sort_values(by='total_inspections', ascending=False)

# Output the result DataFrame
pivot_result.reset_index().head()


risk_category,inspection_type,High Risk,Low Risk,Moderate Risk,total_inspections
0,Routine - Unscheduled,36.0,105.0,72.0,213.0
1,Complaint,1.0,1.0,3.0,5.0
2,Reinspection/Followup,0.0,2.0,2.0,4.0
3,New Ownership,1.0,1.0,1.0,3.0


### 5 -Find the average total compensation based on employee titles and gender. Total compensation is calculated by adding both the salary and bonus of each employee. However, not every employee receives a bonus so disregard employees without bonuses in your calculation. Employee can receive more than one bonus.
Output the employee title, gender (i.e., sex), along with the average total compensation.

In [12]:
sf_employee.columns

Index(['id', 'first_name', 'last_name', 'age', 'sex', 'employee_title',
       'department', 'salary', 'target', 'email', 'city', 'address',
       'manager_id'],
      dtype='object')

In [13]:
sf_bonus.columns

Index(['worker_ref_id', 'bonus'], dtype='object')

In [14]:
import pandas as pd
import numpy as np

sf_bonus_summary = sf_bonus.groupby(['worker_ref_id'])['bonus'].sum().to_frame('bonus').reset_index()
merged_df = pd.merge(sf_employee,sf_bonus_summary,left_on='id',right_on='worker_ref_id')
merged_df['avg_total_comp'] = merged_df['salary'] + merged_df['bonus']
result = merged_df.groupby(['employee_title','sex'])['avg_total_comp'].mean().reset_index()
result.head()

,employee_title,sex,avg_total_comp
0,Auditor,M,2200.0
1,Manager,F,209500.0
2,Sales,M,4600.0
3,Senior Sales,M,5350.0


### 6 -Find the top 2 highest paid City employees for each job title. Use totalpaybenefits column for their ranking. Output the job title along with the corresponding highest and second-highest paid employees.

In [15]:
import pandas as pd

# Sort the DataFrame by job title and total pay benefits
sorted_salaries = sf_public_salaries.sort_values(by=['jobtitle', 'totalpaybenefits'], ascending=[True, False])

# Group by job title and get the top 2 employees for each title
top_employees = sorted_salaries.groupby('jobtitle').head(2).reset_index(drop=True)

# Output the first few rows of the result
top_employees[['jobtitle', 'employeename', 'totalpaybenefits']].head()


,jobtitle,employeename,totalpaybenefits
0,CAPTAIN III (POLICE DEPARTMENT),PATRICIA JACKSON,297608.92
1,CAPTAIN III (POLICE DEPARTMENT),ANNA BROWN,238551.88
2,Deputy Sheriff,Lance A Obtinalla Jr,137519.28
3,Deputy Sheriff,Arnel Maracha,137235.94
4,EMT/Paramedic/Firefighter,Alexander M Lamond,139818.10


### 7 -Get the job titles of the 3 employees who received the most overtime pay
Output the job title of selected records.

In [16]:
import pandas as pd

# Sort the DataFrame by overtime pay in descending order
top_overtime_employees = sf_public_salaries.sort_values(by='overtimepay', ascending=False)

# Select the top 3 employees and get their job titles
result = top_overtime_employees[['jobtitle']].head(3)

result


,jobtitle
195,CAPTAIN III (POLICE DEPARTMENT)
101,Deputy Sheriff
58,EMT/Paramedic/Firefighter


### 8 -Find the employee who earned most from working overtime. Output the employee name.

In [17]:
import pandas as pd

# Find the employee with the highest overtime pay
top_overtime_employee = sf_public_salaries.loc[sf_public_salaries['overtimepay'].idxmax()]

# Output the employee name
result = pd.DataFrame({'employee_name': [top_overtime_employee['employeename']]})

result


,employee_name
0,PATRICIA JACKSON


### 9 -Find the top 5 least paid employees for each job title.
Output the employee name, job title and total pay with benefits for the first 5 least paid employees. Avoid gaps in ranking.

In [19]:
import pandas as pd

# Sort by job title and total pay with benefits
sorted_salaries = sf_public_salaries.sort_values(['jobtitle', 'totalpaybenefits'])

# Group by job title and get the top 5 least paid employees for each
result = sorted_salaries.groupby('jobtitle').head(5)[['employeename', 'jobtitle', 'totalpaybenefits']]

result.reset_index(drop=True, inplace=True)
result.head()


,employeename,jobtitle,totalpaybenefits
0,TERESA BARRETT,CAPTAIN III (POLICE DEPARTMENT),192914.50
1,JOHN LOFTUS,CAPTAIN III (POLICE DEPARTMENT),192951.37
2,DOUGLAS MCEACHERN,CAPTAIN III (POLICE DEPARTMENT),196494.14
3,ANNA BROWN,CAPTAIN III (POLICE DEPARTMENT),238551.88
4,PATRICIA JACKSON,CAPTAIN III (POLICE DEPARTMENT),297608.92


### 10 -Find all people who earned more than the average in 2013 for their designation but were not amongst the top 5 earners for their job title. Use the totalpay column to calculate total earned and output the employee name(s) as the result.

In [22]:
import warnings

# To ignore all warnings
warnings.filterwarnings('ignore')

# If you want to filter specific categories of warnings, you can do it like this:
# warnings.filterwarnings('ignore', category=UserWarning)  # For example, ignore UserWarnings


In [23]:
import pandas as pd

# Filter for 2013 data
data_2013 = sf_public_salaries[sf_public_salaries['year'] == 2013]

# Calculate the average pay for each job title
avg_pay = data_2013.groupby('jobtitle')['totalpay'].mean().reset_index()
avg_pay.columns = ['jobtitle', 'avg_pay']

# Get the top 5 earners for each job title
top_5_earners = data_2013.groupby('jobtitle').apply(lambda x: x.nlargest(5, 'totalpay')).reset_index(drop=True)

# Merge to find those who earned more than average
merged = data_2013.merge(avg_pay, on='jobtitle')
above_average = merged[merged['totalpay'] > merged['avg_pay']]

# Exclude top 5 earners
result = above_average[~above_average['employeename'].isin(top_5_earners['employeename'])]

# Output the employee names
final_result = result[['employeename']].drop_duplicates()

final_result.reset_index(drop=True, inplace=True)
final_result


,employeename
0,Michaela T Womack


### 11 -Find the ratio and the difference between the highest and lowest total pay for each job title.  Another condition is to remove rows total pay equal to zero from the calculation. Output the job title along with the corresponding difference, ratio, highest total pay, and the lowest total pay. Sort records based on the ratio in descending order.

In [24]:
import pandas as pd

# Filter out rows where total pay is zero
filtered_df = sf_public_salaries[sf_public_salaries['totalpay'] > 0]

# Group by job title and calculate highest and lowest total pay
pay_stats = filtered_df.groupby('jobtitle')['totalpay'].agg(['max', 'min']).reset_index()

# Calculate difference and ratio
pay_stats['difference'] = pay_stats['max'] - pay_stats['min']
pay_stats['ratio'] = pay_stats['max'] / pay_stats['min']

# Select relevant columns and sort by ratio in descending order
result = pay_stats[['jobtitle', 'difference', 'ratio', 'max', 'min']].sort_values(by='ratio', ascending=False)

result.head()  # Display the output


,jobtitle,difference,ratio,max,min
14,Registered Nurse,66283.30,11048.216667,66289.30,6.00
19,Transit Operator,27490.96,258.261464,27597.82,106.86
18,Special Nurse,1974.79,53.858405,2012.15,37.36
13,Public Svc Aide-Public Works,4404.45,42.025056,4511.81,107.36
5,Firefighter,23068.79,34.495651,23757.50,688.71


### 12 -Find the median total pay for each job. Output the job title and the corresponding total pay, and sort the results from highest total pay to lowest.

In [25]:
import pandas as pd

# Calculate the median total pay for each job title
median_pay = sf_public_salaries.groupby('jobtitle')['totalpay'].median().reset_index()

# Sort the results from highest to lowest total pay
result = median_pay.sort_values(by='totalpay', ascending=False)

result.head()  # Display the output


,jobtitle,totalpay
6,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,399211.275
0,CAPTAIN III (POLICE DEPARTMENT),196494.140
15,SENIOR PHYSICIAN SPECIALIST,178760.580
17,Sergeant 3,148783.935
1,Deputy Sheriff,95451.055


### 13 -Find the ratio between the number of employees without benefits to total employees. Output the job title, number of employees without benefits, total employees relevant to that job title, and the corresponding ratio. Order records based on the ratio in ascending order.

In [26]:
import pandas as pd

# Count total employees and employees without benefits for each job title
benefits_summary = sf_public_salaries.groupby('jobtitle').agg(
    total_employees=('id', 'count'),
    employees_without_benefits=('benefits', lambda x: (x == 0).sum())
).reset_index()

# Calculate the ratio
benefits_summary['ratio'] = benefits_summary['employees_without_benefits'] / benefits_summary['total_employees']

# Select relevant columns and order by ratio
result = benefits_summary[['jobtitle', 'employees_without_benefits', 'total_employees', 'ratio']].sort_values(by='ratio')

result.head()  # Display the output


,jobtitle,employees_without_benefits,total_employees,ratio
0,CAPTAIN III (POLICE DEPARTMENT),0,5,0.0
17,Sergeant 3,0,8,0.0
16,Senior Eligibility Worker,0,1,0.0
15,SENIOR PHYSICIAN SPECIALIST,0,1,0.0
14,Registered Nurse,0,21,0.0


### 14 -Find the employee who earned the lowest total payment with benefits from a list of employees who earned more from other payments compared to their base pay. Output the first name of the employee along with the corresponding total payment with benefits

In [27]:
import pandas as pd

# Filter employees who earned more from other payments compared to base pay
filtered_employees = sf_public_salaries[sf_public_salaries['totalpaybenefits'] > sf_public_salaries['basepay']]

# Find the employee with the lowest total payment with benefits
lowest_paid_employee = filtered_employees.loc[filtered_employees['totalpaybenefits'].idxmin()]

# Create a DataFrame for the result
result = pd.DataFrame({
    'first_name': [lowest_paid_employee['employeename']],
    'total_payment_with_benefits': [lowest_paid_employee['totalpaybenefits']]
})

result.head()  # Display the output


,first_name,total_payment_with_benefits
0,Renato C Gurion,7.24


### 15 -Find the top 5 highest paid and top 5 least paid employees in 2012.
Output the employee name along with the corresponding total pay with benefits.
Sort records based on the total payment with benefits in ascending order.

In [28]:
import pandas as pd

# Filter for the year 2012
employees_2012 = sf_public_salaries[sf_public_salaries['year'] == 2012]

# Get the top 5 highest and least paid employees
top_paid = employees_2012.nlargest(5, 'totalpaybenefits')[['employeename', 'totalpaybenefits']]
least_paid = employees_2012.nsmallest(5, 'totalpaybenefits')[['employeename', 'totalpaybenefits']]

# Combine the results
result = pd.concat([least_paid, top_paid]).sort_values('totalpaybenefits')

result.reset_index(drop=True, inplace=True)
result.head()  # Display the output


,employeename,totalpaybenefits
0,Grace Salud,46.27
1,Andre Thomas,172.13
2,Connie Love-Miles,332.23
3,Catheryn Williams,474.54
4,Marcus Duty,578.19


### 16 -Find employees who earned the highest and the lowest total pay without any benefits.
Output the employee name along with the total pay.
Order records based on the total pay in descending order.

In [29]:
import pandas as pd

# Filter employees who earned total pay without benefits
employees_no_benefits = sf_public_salaries[sf_public_salaries['totalpaybenefits'] == sf_public_salaries['basepay']]

# Get the highest and lowest total pay employees
highest_paid = employees_no_benefits.loc[employees_no_benefits['totalpay'].idxmax(), ['employeename', 'totalpay']]
lowest_paid = employees_no_benefits.loc[employees_no_benefits['totalpay'].idxmin(), ['employeename', 'totalpay']]

# Combine the results into a DataFrame
result = pd.DataFrame([lowest_paid, highest_paid]).reset_index(drop=True)

# Output the result
result.head()


,employeename,totalpay
0,EDWARD REISKIN,230827.12
1,EDWARD REISKIN,230827.12


### 17 -Find the number of police officers (job title contains substring police), firefighters (job title contains substring fire), and medical staff employees (job title contains substring medical) based on the employee name.
Output each job title along with the corresponding number of employees.

In [30]:
import pandas as pd

# Count police officers, firefighters, and medical staff
police_count = sf_public_salaries[sf_public_salaries['jobtitle'].str.contains('police', case=False)].shape[0]
firefighter_count = sf_public_salaries[sf_public_salaries['jobtitle'].str.contains('fire', case=False)].shape[0]
medical_count = sf_public_salaries[sf_public_salaries['jobtitle'].str.contains('medical', case=False)].shape[0]

# Create a result DataFrame
result = pd.DataFrame({
    'job_title': ['Police Officers', 'Firefighters', 'Medical Staff'],
    'employee_count': [police_count, firefighter_count, medical_count]
})

# Output the result
result.head()


,job_title,employee_count
0,Police Officers,5
1,Firefighters,21
2,Medical Staff,0


### 18 -Find all employees with a job title that contains 'METROPOLITAN TRANSIT AUTHORITY' and output the employee's name along with the corresponding total pay with benefits.

In [31]:
import pandas as pd

# Filter employees with job title containing 'METROPOLITAN TRANSIT AUTHORITY'
result = sf_public_salaries[sf_public_salaries['jobtitle'].str.contains('METROPOLITAN TRANSIT AUTHORITY', case=False)]

# Select relevant columns
output = result[['employeename', 'totalpaybenefits']]

# Output the result
output.head()


,employeename,totalpaybenefits
193,EDWARD REISKIN,230827.12
194,NATHANIEL FORD,567595.43


### 19 -Find benefits that people with the name 'Patrick' have.
Output the full employee name along with the corresponding benefits.

In [32]:
import pandas as pd

# Filter employees with the name 'Patrick'
result = sf_public_salaries[sf_public_salaries['employeename'].str.contains('Patrick', case=False)]

# Select relevant columns
output = result[['employeename', 'benefits']]

# Output the result
output.head()


,employeename,benefits
183,Russell Patrick Mangahas,277.75
184,Patrick F Mcpartland,5.01
185,Patrick Crane,34143.72
186,Patrick Martinez,33194.97
187,Patrick T Truong,31360.09


### 20 -Find the base pay for Police Captains.
Output the employee name along with the corresponding base pay.

In [34]:
import pandas as pd
import numpy as np

result = sf_public_salaries[
    (sf_public_salaries['jobtitle'].str.contains('CAPTAIN', case=False)) & 
    (sf_public_salaries['jobtitle'].str.contains('POLICE', case=False))
][['employeename', 'basepay']]

# Display the first few rows of the result
result.head()


,employeename,basepay
195,PATRICIA JACKSON,99722.00
196,TERESA BARRETT,188328.08
197,ANNA BROWN,102571.24
198,DOUGLAS MCEACHERN,194566.01
199,JOHN LOFTUS,188341.62


### 21 -Find the number of patrons who renewed books at least once but less than 10 times in April 2015. Each row is an unique patron.

In [36]:
library_usage.columns

Index(['patron_type_code', 'patron_type_definition', 'total_checkouts',
       'total_renewals', 'age_range', 'home_library_code',
       'home_library_definition', 'circulation_active_month',
       'circulation_active_year', 'notice_preference_code',
       'notice_preference_definition', 'provided_email_address',
       'year_patron_registered', 'outside_of_county', 'supervisor_district'],
      dtype='object')

In [37]:
import pandas as pd

# Group by patron type and sum checkouts and renewals
summary = library_usage.groupby('patron_type_code')[['total_checkouts', 'total_renewals']].sum().reset_index()

# Display the result
summary.head()


,patron_type_code,total_checkouts,total_renewals
0,0,11612,6392
1,1,4655,309
2,2,4,0
3,3,8216,931
4,4,3,0


### 22 -Find libraries with the highest number of total renewals.
Output all home library definitions along with the corresponding total renewals.
Order records by total renewals in descending order.

In [38]:
# Group by home library definition and sum total renewals
top_libraries = library_usage.groupby('home_library_definition')['total_renewals'].sum().reset_index()

# Sort by total renewals in descending order
top_libraries = top_libraries.sort_values(by='total_renewals', ascending=False)

# Output the result
top_libraries.head()


,home_library_definition,total_renewals
3,Eureka Valley/Harvey Milk Memorial,2421
15,Ortega,2421
17,Parkside,1072
2,Chinatown,410
14,North Beach,256


### 23 -Find the average total checkouts from Chinatown libraries in 2016.

In [39]:
# Filter for Chinatown libraries and year 2016
chinatown_checkouts = library_usage[(library_usage['home_library_definition'].str.contains('Chinatown')) & 
                                     (library_usage['circulation_active_year'] == 2016)]

# Calculate the average total checkouts
average_checkouts = chinatown_checkouts['total_checkouts'].mean()


### 24 - Find months with the highest number of checkouts for main libraries in 2013.
Output the circulation active month along with the corresponding total monthly checkouts.
Order results based on total monthly checkouts in descending order.

In [40]:
# Filter for main libraries and year 2013
main_library_checkouts = library_usage[(library_usage['home_library_definition'].str.contains('Main')) & 
                                        (library_usage['circulation_active_year'] == 2013)]

# Group by month and sum total checkouts
monthly_checkouts = main_library_checkouts.groupby('circulation_active_month')['total_checkouts'].sum().reset_index()

# Sort results by total monthly checkouts in descending order
result = monthly_checkouts.sort_values(by='total_checkouts', ascending=False)
result.head()

,circulation_active_month,total_checkouts
0,April,281
2,July,74
1,January,28
4,October,5
3,June,4


### 25 -Find library types with the highest total checkouts in April made by patrons who had registered in 2015 and whose age was between 65 and 74 years.
Output the year patron registered and the home library definition along with the corresponding highest total checkouts. Sort records based on the highest total checkouts in descending order.

In [42]:
import pandas as pd
import numpy as np

# Filter for patrons aged 65 to 74 years, registered in 2015, and checkouts in April
filtered = library_usage[
    (library_usage['age_range'] == '65 to 74 years') & 
    (library_usage['year_patron_registered'] == 2015) & 
    (library_usage['circulation_active_month'] == 'April')
]

# Group by home library definition and year patron registered, then get the maximum total checkouts
grouped = filtered.groupby(['home_library_definition', 'year_patron_registered'])['total_checkouts'].max()

# Convert the Series to a DataFrame and rename the column
max_checkouts = grouped.to_frame('max_total_checkouts').reset_index()

# Sort the results by maximum total checkouts in descending order
result = max_checkouts.sort_values('max_total_checkouts', ascending=False)
result.head()

,home_library_definition,year_patron_registered,max_total_checkouts
0,Ingleside,2015,38
2,Mission,2015,36
4,Western Addition,2015,25
3,Noe Valley/Sally Brunn,2015,19
1,Main Library,2015,5


### 26 -Find library types with the highest total checkouts made by adults registered in 2010.
Output the year patron registered, home library definition along with the corresponding highest total checkouts

In [43]:
import pandas as pd
import numpy as np

# Filter for adults registered in 2010
adult = library_usage[(library_usage['patron_type_definition'] == 'ADULT') & 
                      (library_usage['year_patron_registered'] == 2010)]

# Group by home library definition and year, then get the maximum total checkouts
grouped = adult.groupby(['home_library_definition', 'year_patron_registered'])['total_checkouts'].max() \
                .to_frame('total_checkouts_number').reset_index()

# Find the maximum value of total checkouts number
max_checkouts = max(grouped['total_checkouts_number'])

# Filter the grouped DataFrame to get the records with the highest total checkouts
result = grouped[grouped['total_checkouts_number'] == max_checkouts]
result.head()

,home_library_definition,year_patron_registered,total_checkouts_number
1,Visitacion Valley,2010,27


### 27 -Find how many people registered in libraries in the year 2016.
Output the total patrons. Keep in mind that each row represents different patron.

In [44]:
total_patrons_2016 = library_usage[library_usage['year_patron_registered'] == 2016].shape[0]


### 28 -Find libraries who haven't provided the email address in circulation year 2016 but their notice preference definition is set to email.
Output the library code.

In [46]:
import pandas as pd
import numpy as np

# Filter for rows where notice preference is 'email', provided email address is False, and circulation year is 2016
filtered = library_usage[
    (library_usage['notice_preference_definition'] == 'email') &
    (library_usage['provided_email_address'] == False) &
    (library_usage['circulation_active_year'] == 2016)
]

# Select only the home library code column
selected_columns = filtered[['home_library_code']]

# Remove duplicate rows
result = selected_columns.drop_duplicates()
result.head()

,home_library_code
95,X
96,R3
97,P7
98,E9


### 29 -Find the number of libraries that had 100 or more of total checkouts in February 2015. Be aware that there could be more than one row for certain library on monthly basis.

In [47]:
import pandas as pd

# Filter for February 2015 and total checkouts of 100 or more
filtered = library_usage[
    (library_usage['circulation_active_month'] == 'February') &
    (library_usage['circulation_active_year'] == 2015) &
    (library_usage['total_checkouts'] >= 100)
]

# Drop duplicates to count each library only once
unique_libraries = filtered['home_library_code'].drop_duplicates()

# Output the result using .head()
result = unique_libraries.head()
result


80    M8
81    G4
82    M4
83     X
Name: home_library_code, dtype: object

### 30 -Find the number of patrons that have made the highest checkouts up to 10 (excluding 10).
Output the number of patrons along with the corresponding total checkouts. Sort records based on the total checkouts in descending order.

In [48]:
import pandas as pd

# Filter for patrons with total checkouts less than 10
filtered = library_usage[library_usage['total_checkouts'] < 10]

# Group by total checkouts and count patrons
result = filtered.groupby('total_checkouts').size().reset_index(name='number_of_patrons')

# Sort by total checkouts in descending order
result = result.sort_values('total_checkouts', ascending=False)

result.head()


,total_checkouts,number_of_patrons
8,9,2
7,8,1
6,6,1
5,5,4
4,4,7


### 31 -Find the most dangerous places in SF based on the crime count per address and district combination.
Output the number of incidents alongside the corresponding address and the district.
Order records based on the number of occurrences in descending order.

In [49]:
sf_crime_incidents_2014_01.columns

Index(['incidnt_num', 'category', 'descript', 'day_of_week', 'date', 'time',
       'pd_district', 'resolution', 'address', 'lon', 'lat', 'location', 'id'],
      dtype='object')

In [50]:
import pandas as pd

# Group by address and district, counting incidents
dangerous_places = sf_crime_incidents_2014_01.groupby(['address', 'pd_district']).size().reset_index(name='incident_count')

# Sort by the number of incidents in descending order
result = dangerous_places.sort_values('incident_count', ascending=False)

result.head()


,address,pd_district,incident_count
67,800 Block of BRYANT ST,SOUTHERN,4
69,800 Block of MARKET ST,SOUTHERN,2
92,TURK ST / TAYLOR ST,TENDERLOIN,2
0,0 Block of 1ST ST,SOUTHERN,1
60,500 Block of PORTOLA DR,INGLESIDE,1


### 32 -Find districts alongside their incidents.
Output the district name alongside the number of incident occurrences.
Order records based on the number of occurrences in descending order.

In [51]:
import pandas as pd

# Group by district and count incidents
district_incidents = sf_crime_incidents_2014_01.groupby('pd_district').size().reset_index(name='incident_count')

# Sort by the number of incidents in descending order
result = district_incidents.sort_values('incident_count', ascending=False)

result.head()


,pd_district,incident_count
7,SOUTHERN,23
4,NORTHERN,18
2,INGLESIDE,11
9,TENDERLOIN,11
3,MISSION,9


### 33 -Find top crime categories in 2014 based on the number of occurrences.
Output the number of crime occurrences alongside the corresponding category name.
Order records based on the number of occurrences in descending order

In [52]:
import pandas as pd

# Group by crime category and count occurrences
category_counts = sf_crime_incidents_2014_01.groupby('category').size().reset_index(name='occurrences')

# Sort by occurrences in descending order
result = category_counts.sort_values('occurrences', ascending=False)

result.head()


,category,occurrences
6,LARCENY/THEFT,31
8,NON-CRIMINAL,11
0,ASSAULT,7
1,BURGLARY,7
16,VEHICLE THEFT,6


### 34 -Find the median inspection score of each business and output the result along with the business name. Order records based on the inspection score in descending order.
Try to come up with your own precise median calculation. In Postgres there is percentile_disc function available, however it's only approximation.

In [53]:
import pandas as pd

# Group by business name and calculate the median inspection score
median_scores = sf_restaurant_health_violations.groupby('business_name')['inspection_score'].median().reset_index()

# Rename the columns for clarity
median_scores.columns = ['business_name', 'median_inspection_score']

# Sort by median inspection score in descending order
result = median_scores.sort_values('median_inspection_score', ascending=False)

result.head()


,business_name,median_inspection_score
47,Castro Street Chevron,100.0
118,Jersey,100.0
163,North Point Market,100.0
61,"Crepe and Brioche, Inc.",100.0
97,Hilton Financial District- Restaurant Seven Fifty,100.0


### 35 -Determine the change in the number of daily violations by calculating the difference between the count of current and previous violations by inspection date.
Output the inspection date and the change in the number of daily violations. Order your results by the earliest inspection date first.

In [54]:
import pandas as pd
import numpy as np
import datetime

# Use the DataFrame
df = sf_restaurant_health_violations

# Convert 'inspection_date' to datetime
df.loc[:, 'inspection_date'] = df['inspection_date'].apply(pd.to_datetime)

# Group by 'inspection_date' and count 'violation_id'
result = df.groupby(['inspection_date'])['violation_id'].count().to_frame('daily_violations_count').reset_index()

# Sort by 'inspection_date'
result = result.sort_values('inspection_date')

# Calculate the difference in daily violations
result['diff'] = result['daily_violations_count'] - result['daily_violations_count'].shift(1)

# Select relevant columns to display
result[['inspection_date', 'diff']]


,inspection_date,diff
0,2015-09-08,NaN
1,2015-09-15,-1.0
2,2015-09-18,0.0
3,2015-09-23,0.0
4,2015-09-28,1.0
...,...,...
222,2018-07-20,-1.0
223,2018-07-24,1.0
224,2018-07-25,-1.0
225,2018-08-02,1.0


### 36 -For every year, find the worst business in the dataset. The worst business has the most violations during the year. You should output the year, business name, and number of violations.

In [55]:
import pandas as pd

# Ensure 'inspection_date' is a datetime column
sf_restaurant_health_violations['inspection_date'] = pd.to_datetime(sf_restaurant_health_violations['inspection_date'])

# Extract the year from 'inspection_date'
sf_restaurant_health_violations['year'] = sf_restaurant_health_violations['inspection_date'].dt.year

# Group by year and business name, counting violations
violations_count = sf_restaurant_health_violations.groupby(['year', 'business_name'])['violation_id'].count().reset_index()

# Find the worst business (most violations) for each year
worst_businesses = violations_count.loc[violations_count.groupby('year')['violation_id'].idxmax()]

# Rename columns for clarity
worst_businesses = worst_businesses.rename(columns={'violation_id': 'number_of_violations'})

# Output the result
worst_businesses[['year', 'business_name', 'number_of_violations']]


,year,business_name,number_of_violations
12,2015,Roxanne Cafe,5
36,2016,Da Cafe,4
187,2017,Peet's Coffee & Tea,2
247,2018,Lollipot,6


### 37 - Verify that the first 4 digits are equal to 1415 for all phone numbers. Output the number of businesses with a phone number that does not start with 1415. It's expected such number should be 0

In [56]:
# Convert phone numbers to string format
sf_restaurant_health_violations['business_phone_number'] = sf_restaurant_health_violations['business_phone_number'].astype(str)

# Check if phone numbers do not start with '1415'
non_matching_count = sf_restaurant_health_violations[~sf_restaurant_health_violations['business_phone_number'].str.startswith('1415')].shape[0]

# Output the count
non_matching_count


215

### 38 -Find details of the business with the highest number of high-risk violations. Output all columns from the dataset considering business_id which consist 'high risk' phrase in risk_category column.

In [57]:
import pandas as pd

# Filter high-risk businesses
high_risk_businesses = sf_restaurant_health_violations[sf_restaurant_health_violations['risk_category'].notna() &
                                                       sf_restaurant_health_violations['risk_category'].str.contains('high risk', case=False)]

# Count violations for each business
result = high_risk_businesses.groupby('business_id').size().reset_index(name='violation_count')

# Get the business with the highest violations
top_business_id = result.loc[result['violation_count'].idxmax(), 'business_id']

# Retrieve all details for that business
business_details = sf_restaurant_health_violations[sf_restaurant_health_violations['business_id'] == top_business_id]

# Output the details
business_details


,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,year
280,86647,Da Cafe,407 Clement St,San Francisco,CA,94118.0,NaN,NaN,NaN,nan,86647_20160907,2016-09-07,48.0,Routine - Unscheduled,86647_20160907_103112,No hot water or running water,High Risk,2016
284,86647,Da Cafe,407 Clement St,San Francisco,CA,94118.0,NaN,NaN,NaN,nan,86647_20160907,2016-09-07,48.0,Routine - Unscheduled,86647_20160907_103109,Unclean or unsanitary food contact surfaces,High Risk,2016
285,86647,Da Cafe,407 Clement St,San Francisco,CA,94118.0,NaN,NaN,NaN,nan,86647_20160907,2016-09-07,48.0,Routine - Unscheduled,86647_20160907_103102,Unclean hands or improper use of gloves,High Risk,2016
286,86647,Da Cafe,407 Clement St,San Francisco,CA,94118.0,NaN,NaN,NaN,nan,86647_20160907,2016-09-07,48.0,Routine - Unscheduled,86647_20160907_103114,High risk vermin infestation,High Risk,2016


### 39 -Find the number of inspections that happened in the municipality with postal code 94102 during January, May or November in each year.
Output the count of each month separately.

In [60]:
import pandas as pd

# Ensure inspection_date is in datetime format
sf_restaurant_health_violations['inspection_date'] = pd.to_datetime(sf_restaurant_health_violations['inspection_date'])

# Filter inspections for postal code 94102 in January, May, or November
filtered_inspections = sf_restaurant_health_violations[
    (sf_restaurant_health_violations['business_postal_code'] == 94102) &
    (sf_restaurant_health_violations['inspection_date'].dt.month.isin([1, 5, 11]))
]

# Extract year and month for grouping
filtered_inspections['year'] = filtered_inspections['inspection_date'].dt.year
filtered_inspections['month'] = filtered_inspections['inspection_date'].dt.month_name()

# Count inspections by year and month
result = filtered_inspections.groupby(['year', 'month']).size().unstack(fill_value=0)

# Ensure January, May, and November are included in the result
result = result.reindex(columns=['January', 'May', 'November'], fill_value=0)

# Reindex to include all years present in the original data
years = sf_restaurant_health_violations['inspection_date'].dt.year.unique()
result = result.reindex(years, fill_value=0)

# Output the result
result


month,January,May,November
year,,,
2017,0,2,0
2016,1,1,1
2018,0,1,0
2015,0,0,0


### 40 -Find the number of complaints that ended in a violation.

In [61]:
import pandas as pd

# Assuming there's a column 'violation_id' that indicates a violation exists
# and 'complaint' is a column indicating if a record is a complaint

# Filter for complaints that ended in a violation
complaints_with_violations = sf_restaurant_health_violations[
    sf_restaurant_health_violations['violation_id'].notna()  # Only include records with violations
]

# Count the number of complaints that ended in a violation
number_of_complaints_with_violations = complaints_with_violations.shape[0]

# Output the result
number_of_complaints_with_violations


225

### 41 -Find the first and last inspections for vermin infestations per municipality.
Output the result along with the business postal code.

In [62]:
import pandas as pd

# Filter for inspections related to vermin infestations
vermin_inspections = sf_restaurant_health_violations[sf_restaurant_health_violations['violation_description'].str.contains('vermin infestation', case=False, na=False)]

# Convert 'inspection_date' to datetime if not already
vermin_inspections['inspection_date'] = pd.to_datetime(vermin_inspections['inspection_date'])

# Sort by inspection date
vermin_inspections = vermin_inspections.sort_values(by='inspection_date')

# Group by postal code and get the first and last inspection dates
result = vermin_inspections.groupby('business_postal_code').agg(
    first_inspection=('inspection_date', 'first'),
    last_inspection=('inspection_date', 'last')
).reset_index()

# Output the result
result


,business_postal_code,first_inspection,last_inspection
0,94102.0,2016-07-20,2018-03-28
1,94103.0,2017-05-05,2018-01-17
2,94108.0,2016-01-07,2016-01-07
3,94110.0,2016-05-02,2018-05-09
4,94111.0,2017-09-26,2017-09-26
5,94112.0,2016-09-07,2016-09-07
6,94114.0,2017-09-27,2018-06-05
7,94115.0,2016-09-19,2016-09-19
8,94118.0,2016-09-07,2016-11-14
9,94133.0,2017-05-23,2017-05-23


### 42 -Find all businesses whose lowest and highest inspection scores are different.
Output the corresponding business name and the lowest and highest scores of each business. HINT: you can assume there are no different businesses that share the same business name
Order the result based on the business name in ascending order.

In [63]:
import pandas as pd

# Group by business name and calculate the min and max inspection scores
scores = sf_restaurant_health_violations.groupby('business_name')['inspection_score'].agg(
    lowest_score='min',
    highest_score='max'
).reset_index()

# Filter businesses where lowest and highest scores are different
result = scores[scores['lowest_score'] != scores['highest_score']]

# Sort by business name
result = result.sort_values('business_name')

# Output the result
result.dropna()


,business_name,lowest_score,highest_score
55,City Super,77.0,78.0
83,"Fresca Gardens, Inc",82.0,88.0
119,Jiang Ling Cuisine Restaurant,72.0,74.0
175,Peet's Coffee & Tea,94.0,96.0
182,Project Juice,96.0,100.0
195,Roxanne Cafe,86.0,92.0
228,Sutter Pub and Restaurant,88.0,90.0
260,Wines of California Wine Bar,88.0,92.0


### 43 -Count the number of inspections per each risk category.
Categorize records with null values under the 'No Risk' category.

Sort the result based on the number of inspections in descending order.

In [64]:
import pandas as pd

# Replace null risk categories with 'No Risk'
sf_restaurant_health_violations['risk_category'] = sf_restaurant_health_violations['risk_category'].fillna('No Risk')

# Count inspections per risk category
result = sf_restaurant_health_violations['risk_category'].value_counts().reset_index()
result.columns = ['risk_category', 'inspection_count']

# Sort by inspection count in descending order
result = result.sort_values('inspection_count', ascending=False)

# Output the result
result


,risk_category,inspection_count
0,Low Risk,109
1,Moderate Risk,78
2,No Risk,72
3,High Risk,38


### 44 -You're given a dataset of health inspections. Count the number of violation in an inspection in 'Roxanne Cafe' for each year. If an inspection resulted in a violation, there will be a value in the 'violation_id' column. Output the number of violations by year in ascending order.

In [65]:
import pandas as pd

# Filter for 'Roxanne Cafe' and non-null violation_ids
roxanne_violations = sf_restaurant_health_violations[
    (sf_restaurant_health_violations['business_name'] == 'Roxanne Cafe') &
    (sf_restaurant_health_violations['violation_id'].notna())
]

# Extract year from inspection_date and count violations
roxanne_violations['year'] = roxanne_violations['inspection_date'].dt.year
violations_count = roxanne_violations.groupby('year')['violation_id'].count().reset_index()

# Rename columns
violations_count.columns = ['year', 'violation_count']

# Sort by year in ascending order
violations_count = violations_count.sort_values('year')

# Output the result
violations_count


,year,violation_count
0,2015,5
1,2016,2
2,2018,3


### 45 -Find the number of violations that each school had. Any inspection is considered a violation if its risk category is not null.
Output the corresponding business name along with the result.
Order the result based on the number of violations in descending order.

In [66]:
import pandas as pd

# Filter for schools with non-null risk categories
school_violations = sf_restaurant_health_violations[sf_restaurant_health_violations['risk_category'].notna()]

# Group by business name and count violations
violations_count = school_violations.groupby('business_name').size().reset_index(name='violation_count')

# Sort by violation count in descending order
violations_count = violations_count.sort_values('violation_count', ascending=False)

# Output the result
violations_count


,business_name,violation_count
195,Roxanne Cafe,10
137,Lollipot,6
64,Da Cafe,4
175,Peet's Coffee & Tea,3
119,Jiang Ling Cuisine Restaurant,2
...,...,...
96,Heritage,1
97,Hilton Financial District- Restaurant Seven Fifty,1
98,Home Plate,1
99,Hong Kee & Kim,1


### 46 -Classify each business as either a restaurant, cafe, school, or other.


•	A restaurant should have the word 'restaurant' in the business name.
•	A cafe should have either 'cafe', 'café', or 'coffee' in the business name.
•	A school should have the word 'school' in the business name.
•	All other businesses should be classified as 'other'.


Output the business name and their classification.

In [67]:
import pandas as pd

# Select relevant columns
result = sf_restaurant_health_violations.iloc[:, :2]

# Classify businesses based on the name
result['business_type'] = result['business_name'].apply(
    lambda x: 'school' if 'school' in x.lower() 
    else 'restaurant' if 'restaurant' in x.lower() 
    else 'cafe' if 'cafe' in x.lower() or 'coffee' in x.lower() or 'café' in x.lower()
    else 'other'
)

# Select relevant columns and drop duplicates
result = result[['business_name', 'business_type']].drop_duplicates()


### 47 -Find the postal code which has the highest average inspection score.
Output the corresponding postal code along with the result.

In [68]:
import pandas as pd

# Group by postal code and calculate the average inspection score
average_scores = sf_restaurant_health_violations.groupby('business_postal_code')['inspection_score'].mean()

# Find the postal code with the highest average score
highest_postal_code = average_scores.idxmax()
highest_score = average_scores.max()

# Create the result DataFrame
result = pd.DataFrame({'business_postal_code': [highest_postal_code], 'average_inspection_score': [highest_score]})

result


,business_postal_code,average_inspection_score
0,94132.0,92.5


### 48 -Find all inspections made on restaurants and output the business name and the inspection score. For this question business is considered as a restaurant if it contains string "restaurant" inside its name.

In [70]:
import pandas as pd

# Filter for inspections on restaurants
restaurant_inspections = sf_restaurant_health_violations[sf_restaurant_health_violations['business_name'].str.contains('restaurant', case=False)]

# Select relevant columns
result = restaurant_inspections[['business_name', 'inspection_score']]

result.head()


,business_name,inspection_score
1,Sutter Pub and Restaurant,88.0
3,Washington Bakery & Restaurant,67.0
4,Brothers Restaurant,79.0
8,Jiang Ling Cuisine Restaurant,72.0
9,Wing Lee BBQ Restaurant,78.0


### 49 -Find the business names that scored less than 50 in inspections.
Output the result along with the corresponding inspection date and the score.

In [71]:
import pandas as pd

# Filter for businesses with inspection scores less than 50
low_scores = sf_restaurant_health_violations[sf_restaurant_health_violations['inspection_score'] < 50]

# Select relevant columns
result = low_scores[['business_name', 'inspection_date', 'inspection_score']]

result.head()


,business_name,inspection_date,inspection_score
230,Lollipot,2018-05-22,45.0
278,Lollipot,2018-05-22,45.0
279,Lollipot,2018-05-22,45.0
280,Da Cafe,2016-09-07,48.0
281,Lollipot,2018-05-22,45.0


### 50 -Find all business postal codes of restaurants with issues related to the water (violation description contains substring "water").

In [72]:
import pandas as pd
import numpy as np

# Filter for violations related to water
water = sf_restaurant_health_violations[sf_restaurant_health_violations['violation_description'].str.contains('water', na=False)]

# Select and drop duplicates for business postal codes
result = water['business_postal_code'].drop_duplicates()


In [73]:
result

166    94133.0
179    94115.0
185    94103.0
210    94118.0
Name: business_postal_code, dtype: float64

### 51 -Find all businesses which have a phone number.

In [74]:
import pandas as pd
import numpy as np

# Filter businesses that have a phone number
phone_number = sf_restaurant_health_violations[sf_restaurant_health_violations['business_phone_number'].notnull()]

# Select unique business names with phone numbers
result = phone_number['business_name'].drop_duplicates()
result.head()

0       John Chin Elementary School
1         Sutter Pub and Restaurant
2                  SRI THAI CUISINE
3    Washington Bakery & Restaurant
4               Brothers Restaurant
Name: business_name, dtype: object

### 52 -Find all businesses which have low-risk safety violations.

In [75]:
import pandas as pd

# Filter for low-risk safety violations
low_risk = sf_restaurant_health_violations[sf_restaurant_health_violations['risk_category'].str.contains('low risk', case=False, na=False)]

# Select relevant columns
result = low_risk[['business_name', 'risk_category']].drop_duplicates()
result.head()

,business_name,risk_category
0,John Chin Elementary School,Low Risk
2,SRI THAI CUISINE,Low Risk
6,"Antonelli Brothers Meat, Fish, and Poultry Inc.",Low Risk
7,STARBUCKS COFFEE CO. #603,Low Risk
9,Wing Lee BBQ Restaurant,Low Risk
